# Main Notebook for Extracting Alcohol Information from PDF Pen Pro Results

In [778]:
import pandas as pd
import numpy as np

import os
import docx
import nltk
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = 'C:\\Users\\layto\\AppData\\Local\\Tesseract-OCR\\tesseract'

docxPath = 'C:\\Users\\layto\\SampleWineCatalogs\\Docx_Files'
jpgPath = 'C:\\Users\\layto\\SampleWineCatalogs\\JPG_Files'

In [779]:
len(os.listdir(docxPath))

85

In [780]:
len(os.listdir(jpgPath))

100

In [781]:
temp = [x.split('.')[0][-4:] for x in os.listdir(docxPath)]
jpg_equal_docx = []
for i in range(len(os.listdir(jpgPath))):
    if os.listdir(jpgPath)[i].split('.')[0][-4:] in temp:
        jpg_equal_docx.append(os.listdir(jpgPath)[i])
    

In [782]:
print(len(jpg_equal_docx))
print(jpg_equal_docx[:5])

85
['UCD_Lehmann_0011.jpg', 'UCD_Lehmann_0014.jpg', 'UCD_Lehmann_0047.jpg', 'UCD_Lehmann_0069.jpg', 'UCD_Lehmann_0190.jpg']


-------------------

In [783]:
def whichIsMore(self):
    '''
    Input a list of strings (for example if you split the textual string on whitespace)
    Returns tuple of the counts of '\n' and '\t' respectively to determine which is better to split on.
    '''
    if self and isinstance(self,list) is True:
        counterN = 0
        counterT = 0
        for line in range(len(self)):
            if self[line].find('\n') != -1:
                counterN += 1
            elif self[line].find('\t') != -1:
                counterT += 1
            elif self[line].find('\n') != -1 and self[line].find('\t') != -1:
                counterN += 1
                counterT += 1
        result = (counterN,counterT)
        if result[0] > result[1]:
            return({'\n':result[0],'by':result[0]-result[1]})
        elif result[1] > result[0]:
            return({'\t':result[1],'by':result[1]-result[0]})
        elif result[0] == 0 and result[1] == 0:
            return(None)
    else:
        raise AttributeError('Input a list of strings')

-----

In [784]:
class wordDocx():
    
    def __init__(self, path):
        self.path = path
    
    def fileList(path):
        '''
        Generates full file paths to Docx files given relative directory input
        '''
        import os
        fullPathFiles = []
        file = os.listdir(path)
        for doc in range(len(file)):
            fullPathFiles.append(path + '\\' + file[doc])
        return(fullPathFiles)
    

    def parseTable(file):
        '''
        When getAllParagraphs function below cannot find any text, it checks for instance of a table in the docx file.
        '''
        from docx.api import Document
        document = Document(file)
        
        data = []
        
        table = document.tables
        for tab in range(len(table)):
            oneTable = table[tab]
            for i, row in enumerate(oneTable.rows):
                text = [cell.text for cell in row.cells]
                newText = [' '.join([x for x in text])]
                data.append([' '.join([x.split() for x in newText][0])])

        return([' '.join(x) for x in data])
    
    
    def getAllParaAndTables(files):
        '''
        Extracts all paragraphs and tables from all Docx files of the PDF Pen Pro Results
        '''
        import docx
        if isinstance(files,list) is True:
            docPara = []
            for file in files:
                iterDoc = docx.Document(file)
                docText = []
                if len(''.join([i.text for i in iterDoc.paragraphs])) > 0 and len(iterDoc.tables) > 0:
                    table = wordDocx.parseTable(file)
                    text = ' '.join([i.text for i in iterDoc.paragraphs])
                    makeLines = text.split(' ')
                    if whichIsMore(makeLines) == None:
                        docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in text.split()])]},{'TABLE':table}]})
                    else:
                        if list(whichIsMore(makeLines).items())[0][0] == '\n':
                            newLines = text.split('\n')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':table}]})
                        elif list(whichIsMore(makeLines).items())[0][0] == '\t':
                            newLines = text.split('\t')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':table}]})
    
                elif len(''.join([i.text for i in iterDoc.paragraphs])) == 0 and len(iterDoc.tables) > 0:              
                    table = wordDocx.parseTable(file)
                    if len(table) > 0:
                        docPara.append({file.split('\\')[-1]:[{'TEXT':None},{'TABLE':table}]})
                    else:
                        return(None)
                elif len(''.join([i.text for i in iterDoc.paragraphs])) > 0 and len(iterDoc.tables) == 0:
                    text = ' '.join([i.text for i in iterDoc.paragraphs])
                    makeLines = text.split(' ')
                    if whichIsMore(makeLines) == None:
                        docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in text.split()])]},{'TABLE':None}]})
                    else:
                        if list(whichIsMore(makeLines).items())[0][0] == '\n':
                            newLines = text.split('\n')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':None}]})
                        elif list(whichIsMore(makeLines).items())[0][0] == '\t':
                            newLines = text.split('\t')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':None}]})
                        
                elif ''.join([i.text for i in iterDoc.paragraphs]) == '' and len(iterDoc.tables) == 0:
                    docPara.append({file.split('\\')[-1]:None})

            return(docPara)

        else:
            raise AttributeError('Needs list of full file paths to the .docx files')

---------

In [842]:
class identifyFormat():
    
    def __init__(self, pargraph):
        self.paragraph = paragraph
        
    def years(mini,maxi):
        '''
        Generates a set of guess year ranges for alcohols
        '''
        import numpy as np
        years = set([n for n in np.arange(mini,maxi,1)])
        return(years)
    
    def wordsAndNums(self):
        '''
        checks word for letters and nums
        '''
        import string
        import numpy as np
        
        setLetters = set(list(string.ascii_lowercase))
        setNums = set([str(x) for x in np.arange(0,10,1)])
        words = set(list(self.lower())).intersection(setLetters)
        nums = set(list(self)).intersection(setNums)
        if len(words) > 1 and len(nums) > 1:
            return('both')
        else:
            return('fine')
        
    def fixLettersNums(self):
        '''
        fixed string for letters and nums together
        '''
        import string
        import numpy as np

        setLetters = set(list(string.ascii_lowercase))
        setNums = set([str(x) for x in np.arange(0,10,1)])
        l = list(self)
        nums = []
        letters = []
        for x,y in enumerate(l):
            if l[x] in setNums:
                nums.append(x)
            elif l[x] in setLetters:
                letters.append(x)
        return(nums,letters)
    
    def splitWord(self):
        '''finds the split of conjoined words'''
        import nltk
        # UNCOMMENET BELOW IF NOT INSTALLED
        #nltk.download('words')
        from nltk.corpus import words
        import string
        split = self.split()
        for i in range(len(split)):
            l = list(split[i])
            upper = [x for x in list(split[i]) if x.isupper() is True]
            if len(upper) == 1:
                word1 = ''.join(l[:l.index(upper[0])]).lower()
                word2 = ''.join(l[l.index(upper[0]):]).lower()
                if len(word1) > 1 and len(word2) > 1:
                    if word1[-1] != 's' and word2[-1] != 's':
                        if word1 in words.words() and word2 in words.words():
                            split[i] = (word1 + ' ' + word2)
                        else:
                            if word1 in words.words() and word2 not in words.words():
                                split[i] = word1
                            elif word1 not in words.words() and word2 in words.words():
                                split[i] = word2
                    else:
                        if word1 in words.words() or word2 in words.words():
                            split[i] = (word1 + ' ' + word2)
        return(' '.join(split))
            
    def removeSpecial(self):
        '''
        Cleans the strings from any special or unnecessary characters
        '''
        special = set([',','"','|','\\',':',';','`','~','#','°','•'])
        split = self.split(' ')
        split = [x for x in split if len(x) > 0]
        fixed = []
        
        for i in range(len(split)):
            test = list(split[i])
            if len(set(test).intersection(special)) == 0 and '.' not in test:
                if 0 in [x.find('(') for x in test]:
                    try:
                        if test[-2] == '(' and test[-1] == ')':
                            split[i] = ''.join(test[:-2])
                            fixed.append(split[i])
                        elif test[0] == '(' and test[1] == ')':
                            split[i] = ''.join(test[2:])
                            fixed.append(split[i])
                        else:
                            fixed.append(split[i])
                    except IndexError:
                        continue
                else:
                    fixed.append(split[i])
            elif test[-1] == '.' and test[0] != '.' and len(test) > 1:
                test = test[:-1]
                split[i] = ''.join(test)
                fixed.append(split[i])
            elif '.' in test:
                if len([x for x in test if x == '.']) == 1:
                    try:
#                         if identifyFormat.wordsAndNums(split[i]) == 'both':
#                             if '$' in test and '.' in test:
#                                 fixed.append(' '.join(split[i]).split('$'))
#                             elif '.' in test and '$' not in test:
#                                 indices = identifyFormat.fixLettersNums(split[i])
#                                 nums = indices[0]
#                                 word = indices[1]
#                                 if sum(nums) > sum(word) and len(word) > 0:
#                                     s1 = split[i][nums[0]:nums[-1]+1]
#                                     s2 = split[i][word[0]-1:word[-1]+1]
#                                     split[i] = s2 + ' ' + s1
#                                     fixed.append(split[i])
#                                 elif sum(nums) < sum(word) and len(nums) > 0:
#                                     s1 = split[i][nums[0]-1:nums[-1]+1]
#                                     s2 = split[i][word[0]:word[-1]+1]
#                                     split[i] = s2 + ' ' + s1
#                                     fixed.append(split[i])
#                             else:
#                                 fixed.append(split[i])
                        
                        if isinstance(int(''.join([x for x in test if x != '.'])),int):
                            split[i] = ''.join(test)
                            fixed.append(split[i])
                        else:
                            split[i] = ''.join([x for x in test if x != '.'])
                            fixed.append(split[i])
                            
                    except ValueError:
                        continue
                
                # For multiple leading '.' before prices
                elif len([y for y in [x.find('.') for x in test] if y == 0]) != 1:
                    tryInt1 = ''.join(test).split('.')[-1]
                    tryInt2 = ''.join(test).split('.')[-2]
                    try:
                        if isinstance(int(tryInt1),int) and isinstance(int(tryInt2),int):
                            if test[0] != '.':
                                joinSplit = ''.join(test).split('.')
                                name = joinSplit[0]
                                prices = joinSplit[-2] + '.' + joinSplit[-1]
                                split[i] = name + ' ' + prices
                                fixed.append(split[i])
                            elif test[0] == '.':
                                joinSplit = ''.join(test).split('.')
                                price = joinSplit[-2] + '.' + joinSplit[-1]
                                split[i] = price
                                fixed.append(split[i])
                    except ValueError:
                        length = len([y for y in [x.find('.') for x in test] if y == 0])-1
                        test = test[length:]
                        split[i] = ''.join(test)
                        fixed.append(split[i])
            else:
                while len(set(test).intersection(special)) > 0:
                    ind = [test.index(x) for x in set(test).intersection(special)]
                    if len(ind) > 0:
                        try:
                            for index in ind:
                                test.pop(index)
                        except IndexError:
                            continue
                    split[i] = ''.join(test)
                    if len(set(split[i]).intersection(special)) == 0:
                        fixed.append(split[i])
                        break
                    else:
                        continue
                
        fixed = ' '.join([''.join(x) for x in fixed])
        return(fixed)
    
    
    def fixBottlePrices(self):
        '''
        Removes special using function above, and fixes bad bottle prices
        '''
        fixed = identifyFormat.removeSpecial(self)
        again = identifyFormat.splitWord(fixed)
        
        preSplit = again.split(' ')
        split = [x for x in preSplit if len(x) > 0]
        
        setRange = np.arange(1,5000,1)
        
        for i in range(len(split)):
            if '.' in list(split[i]) and i >= 2:
                tryInt = split[i-1]
                intList = list(tryInt)
                try:
                    if isinstance(int(tryInt),int):
                        getLength = len(intList)
                        if getLength == 1:
                            split[i-1] = intList[0]+'.'+'00'
                        elif getLength == 2:
                            split[i-1] = intList[0]+'.'+intList[1]+'0'
                        elif getLength == 3:
                            split[i-1] = intList[0]+'.'+intList[1]+intList[2]
                        elif getLength == 4 and int(tryInt) not in np.arange(1880,2000,1):
                            split[i-1] = intList[0]+intList[1]+'.'+intList[2]+intList[3]
                        else:
                            split[i-1] = split[i-1]
                except ValueError:
                    continue
                    
        for i in range(len(split)+1):
            try:
                single = split[i]
                testNO = split[i-1]
                try:
                    if int(single) in np.arange(1,10,1):
                        if testNO.lower() != 'no':
                            ind = split.index(single)
                            split.pop(int(ind))
                except ValueError:
                    continue
            except IndexError:
                break

        return(' '.join(split))
    
                                
    def findYears(self,mini,maxi):
        '''
        finds year(s) in fixed word docx paragraphs associated with respective alcohol
        '''
        yearSet = identifyFormat.years(mini,maxi)
        fixed = identifyFormat.removeSpecial(self)
        split = fixed.split(' ')
        y = []
        for i,j in enumerate(split):
            try:
                year = int(split[i])
                if year and year in yearSet:
                    y.append({str(year):i})
            except ValueError:
                continue
                
        if len(y) > 0:
            return(y)
        else:
            return(0)

    def wordBottle(self):
        '''
        From string of paragraph as input, finds instance of word 'bottle'
        '''
        try:
            test = [x.lower() for x in self.split(' ')]
            find = test.index('bottle')
            if find:
                return(True)
        except ValueError:
            return(False)
        
    def idNum(self):
        '''
        Generates a set of bin ranges for alcohols
        '''
        fixed = self.split(' ')
        import numpy as np
        set1 = set([n for n in np.arange(1,1880,1)])
        set2 = set([n for n in np.arange(2005,15000,1)])
        ID = set(set1.union(set2))
        potentialIDs = []

        for i in fixed:
            try:
                if int(i) in ID:
                    ind = fixed.index(i)
                    potentialIDs.append({i:ind})
            except ValueError:
                continue
            
        if len(potentialIDs) > 0:
            return(potentialIDs)
        elif len(potentialIDs) == 0:
            return('')

----

In [843]:
allData = wordDocx.getAllParaAndTables(wordDocx.fileList(docxPath))

In [844]:
# Cleans "allData" with identifyFormat class

files = wordDocx.fileList(docxPath)
for i in range(len(allData)):
    fileName = files[i].split('\\')[-1]
    if allData[i][fileName] != None:
        preText = allData[i][fileName][0]['TEXT']
        preTable = allData[i][fileName][1]['TABLE']
        if preText != None and preTable != None:
            preText = preText[0]
            preTab = ' '.join([x for x in preTable])
            newText = identifyFormat.fixBottlePrices(preText)
            newTable = identifyFormat.fixBottlePrices(preTab)
            allData[i][fileName][0]['TEXT'] = [newText]
            allData[i][fileName][1]['TABLE'] = [newTable]
        elif preText == None and preTable != None:
            preTable = ' '.join([x for x in preTable])
            newTable = identifyFormat.fixBottlePrices(preTable)
            allData[i][fileName][1]['TABLE'] = [newTable]
        elif preText != None and preTable == None:
            preText = preText[0]
            newText = identifyFormat.fixBottlePrices(preText)
            allData[i][fileName][0]['TEXT'] = [newText]
    else:
        allData[i][fileName] == None

In [845]:
# EXAMPLE
print(len(allData))
allData

85


[{'UCD_Lehmann_0011.docx': [{'TEXT': ['The Great 1961 GLAR ET8 We quote from the new wine classic the Time-Life book on Wines and Spirits 1961 RED BORDEAUX-one of the best years of the century the equal of 1945 Unfortunately the crop was about half the normal size The grapes were well-nourished resulting in concentrated wines rich in every quality including color body bouquet and fruit They are the most long-lived wines of our generation in the classic tradition 20 out of a possible 20 very very great" There is little that we can add to this other than to say that the 1961s have gone up 40% in Bordeaux during the last year if one is fortunate enough to be able to obtain any of the great ones at all 1961 VINTAGE No Bottle Case 237 CHATEAU BELAIR (St Emilion) 4.49 48.50 There are 12 First Great Growths in the 1955 St Emilion official classification Chateau Belair is one of them The vineyard is owned by Chateau Ausone the vines are adjacent and the wine is matured in the Ausone cellar The

In [846]:
# Grabbing tester from the table of UCD_Lehmann_3479.docx 
for i in range(len(allData)):
    try:
        if allData[i]['UCD_Lehmann_3479.docx']:
            t = allData[i]['UCD_Lehmann_3479.docx']
            _3479 = t[0]['TEXT'][0]
            break
    except KeyError:
        continue

In [847]:
_3479

"1966 MoseIles On Sale 1966 Rhines 0 Sale No 551 MOSELBLUMCHEN Kendermann Fresh light ideal summer wine 718 PIESPORTER GOLDTROPFCHEN Veit Delimte fragrant fruity - from best vineyard 633 SCHARZHOFBERGER Egon Muller . Our favorite Moselle Great distinction very light bone-dry 715 GRAACHER HIMIVIELREICH Joh Jos Prum 32.30 Another great very dry Moselle from impeccable origin 632 WEHLENER SONNENUHR Joh Jos Prum Glorious harmony of richness and finesse Distinctly dry 602 MAXIMIN GRUENHAUSER HERRENBERG ABTSBERG von Schubert 3.49 37.70 One of the great vineyards of the Rower - a classic Moselle 592 BRAUNEBERGER JUFFER SPATLESE von Schorlemer 3.49 37.70 Thomas Jefferson spoke about this vineyard as 'the Moselle's best ' Full-bodied 599 BERNCASTELER SCHLOSSBERG SPATLESE Dr H Thanisch 3.49 37.70 Any wine grown by Dr Thanisch represents a jewel of the Moselle 39.85 640 SCHARZHOFBERGER SPATLESE Egon Muller 3.99 43.10 40.95 A combination of steely austerity combined with elegance and fruit Can tak

In [848]:
# Grabbing tester from the table of UCD_Lehmann_0800.docx 
for i in range(len(allData)):
    try:
        if allData[i]['UCD_Lehmann_0800.docx']:
            t2 = allData[i]['UCD_Lehmann_0800.docx']
            _0800 = t2[1]['TABLE'][0]
            break
    except KeyError:
        continue

In [849]:
tester_0800 = identifyFormat.fixBottlePrices(_0800)
tester_0800

"ADVANCE SALE OFFERING OF 1970 URINES & MOSELLES EVERYONE IS TALKING about 1970 on the Rhine and Moselle We have gone to great effort to see that the '70s would be shipped express so they would be here in time to add their freshness fruit and good cheer to your summer pleasures All of the 1970's shown below will reach our cellars the first few days of July and you can count on delivery before July 10th whether it be your apartment in the city or your summer vacation address The growers along the Rhine and Moselle rivers are rightfully happy with their 1970 crop It is true that they did not produce too many peak wines in the exalted 'Auslese' and 'Beerenauslese' class—however this was more than compensated for by a bounteous crop of truly superb typical and well-balanced regionals and the less complex estatebottlings After all it is these less costly and less sweet German wines that have proven most popular in America All of these represent regional wines at their best and come from an 

In [850]:
identifyFormat.findYears(_3479,1800,2000)

[{'1966': 0}, {'1966': 4}, {'1959': 280}, {'1959': 294}]

In [851]:
[list(x.keys())[0] for x in identifyFormat.idNum(_3479)]

['551',
 '718',
 '633',
 '715',
 '632',
 '602',
 '592',
 '599',
 '640',
 '628',
 '618',
 '725',
 '726',
 '590',
 '646',
 '677',
 '511',
 '538',
 '542',
 '737',
 '513',
 '514',
 '524',
 '132',
 '577',
 '561',
 '631',
 '549',
 '583',
 '713',
 '557',
 '702',
 '530',
 '690',
 '578',
 '709',
 '21']

------

## First finalized method for extacting names from tables/paragraphs that use the bottle number, name, year, and price format.

In [852]:
def priceIndexing(para):
    text = para.split(' ')
    indices = []
    for i,j in enumerate(text):
        try:
            if j.find('.') != -1 and isinstance(int(''.join(j.split('.'))),int):
                indices.append(i)
        except ValueError:
            continue

    final = []
    ind1 = 0
    for r in range(len(indices)):
        try:
            if indices[r+1] - indices[r] == 1:
                ind2 = indices[r+1] + 1
                final.append(' '.join(text[ind1:ind2]))
                ind1 = ind2
        except IndexError:
            continue
    return(final)
    
def finalForm(para):
    '''
    input is a textual paragraph preprocessed using the identifyFormat.fixBottlePrices() method which includes removing 
    problematic symbols or characters (via identifyFormat.removeSpecial())
    '''
    findYears = identifyFormat.findYears(para,1880,2000)
    findIDs = identifyFormat.idNum(para)
    newFinal = []
    
    if len(findIDs) > 0: #and len(findIDs) == len(findYears):
        final = priceIndexing(para)
        for i,j in enumerate(final):
            if len(j.split()) > 1:
                if identifyFormat.idNum(j) != None and len(identifyFormat.idNum(j)) == 1 and len(j.split()) < 27:
                    newFinal.append(j)
                elif identifyFormat.idNum(j) != None and len(identifyFormat.idNum(j)) > 1:
                    length = len(identifyFormat.idNum(j))
                    c = 0
                    splitSentence = j.split()
                    for i in range(len(splitSentence)):
                        if splitSentence[i] in [list(x.keys())[0] for x in identifyFormat.idNum(j)]:
                            c += 1
                            if c == length:
                                ind = j.split().index(j.split()[i])
                                j = ' '.join(j.split()[ind:])
                                newFinal.append(j)
#     elif len(findIDs) > 0 and len(findIDs) != len(findYears):

        n = [str(x) for x in np.arange(0,1000,1)]
        for f in range(len(newFinal)):
            if newFinal[f].split()[0] not in n:
                ind = [newFinal[f].split().index(x) for x in newFinal[f].split() if x in n]
                if len(ind) > 0:
                    reconstruct = ' '.join(newFinal[f].split()[ind[0]:])
                    newFinal[f] = reconstruct

#     elif len(findYears) > 0 and len(findIDs) < len(findYears):
#         final = priceIndexing(para)
#         newFinal = []
#         for i,j in enumerate(final):
#             if len(j.split()) > 1:
                


        if len(newFinal) > 0:
            return(newFinal)
        else:
            return('')


In [853]:
finalForm(_3479)

['602 MAXIMIN GRUENHAUSER HERRENBERG ABTSBERG von Schubert 3.49 37.70',
 '592 BRAUNEBERGER JUFFER SPATLESE von Schorlemer 3.49 37.70',
 '599 BERNCASTELER SCHLOSSBERG SPATLESE Dr H Thanisch 3.49 37.70',
 '640 SCHARZHOFBERGER SPATLESE Egon Muller 3.99 43.10',
 '618 MAXIMIN GRUENHAUSER HERRENBERG SPATLESE von Schubert 4.25 45.90',
 '725 PIESPORTER GOLDTROPFCHEN AUSLESE Bischoefliches Konvikt 4.75 51.30',
 '590 BERNCASTELER DOCTOR SPATLESE CABINET Dr H Thanisch 6.99 75.50',
 '511 LMBFRAUMILCH Kendermann 1.49 16.10',
 '542 NIERSTEINER DOMTHAL Kendermann 1.69 18.25',
 '513 ELTVILLER SONNENBERG Langwerth von Simmem 2.25 24.30',
 '514 ELTVILLER SONNENBERG CABINET Langwerth von Simmem 2.79 30.15',
 '524 JOHANNISBERGER KLAUS CABINET Langraf von Hessen 2.79 30.15',
 '132 SCHLOSS VOLLRADS SCHLOSSABZUG Graf Matuschka 2.99 32.30',
 '577 SCHLOSS JOHANNISBERGER ROTLACK Fuerst von Metternich 2.99 32.30',
 '561 RUDESHEIMER BERG ROSENECK CABINET Schloss Schoenborn 2.99 32.30',
 '631 FORSTER JESUTTENGARTE

In [854]:
resultBinNameYearPrices = {}
            
files = wordDocx.fileList(docxPath)
for i in range(len(allData)):
    fileName = files[i].split('\\')[-1]
    if allData[i][fileName] != None:
        preText = allData[i][fileName][0]['TEXT']
        preTable = allData[i][fileName][1]['TABLE']
        if preText != None and preTable != None:
            preText = preText[0]
            preTab = ' '.join([x for x in preTable])
            textResult = finalForm(preText)
            tableResult = finalForm(preTab)
            if textResult != None and tableResult != None:
                if textResult != '' and tableResult != '':
                    resultBinNameYearPrices[fileName] = [{'textResult':textResult},{'tableResult':tableResult}]
                elif textResult == '' and tableResult != '':
                    resultBinNameYearPrices[fileName] = [{'textResult':None},{'tableResult':tableResult}]
                elif textResult != '' and tableResult == '':
                    resultBinNameYearPrices[fileName] = [{'textResult':textResult},{'tableResult':None}]
            
        elif preText == None and preTable != None:
            preTab = ' '.join([x for x in preTable])
            tableResult = finalForm(preTab)
            if tableResult != None and tableResult != '':
                resultBinNameYearPrices[fileName] = [{'textResult':None},{'tableResult':tableResult}]
        
        elif preText != None and preTable == None:
            preText = preText[0]
            textResult = finalForm(preText)
            if textResult != None and textResult != '':
                resultBinNameYearPrices[fileName] = [{'textResult':textResult},{'tableResult':None}]


In [855]:
textual = []
textOnly = [x[1][0]['textResult'] for x in list(resultBinNameYearPrices.items()) if x[1][0]['textResult'] != None]
for i in range(len(textOnly)):
    [textual.append(x) for x in textOnly[i]]

table = []
tableOnly = [x[1][1]['tableResult'] for x in list(resultBinNameYearPrices.items()) if x[1][1]['tableResult'] != None]
for i in range(len(tableOnly)):
    [table.append(x) for x in tableOnly[i]]

In [856]:
format1 = [x for x in table + textual if len(x.split()) < 23 and len(x.split()) > 3]

In [861]:
format1

['753 CHATEAU GISCOURS (Margaux) New ownership-vast improvement-excellent SALE PRICE Bottle Case 2.79 30.75',
 '713 CHATEAU LASCOMBES (Margaux) 2.99 32.95',
 '706 CHATEAU NENIN (Pomerol) 2.99 32.95',
 '757 CHATEAU MALESCOT (Margaux) 2.99 32.95',
 '751 CHATEAU DUCRU BEAUCAILLOU (St Julien) 3.29 36.25',
 '708 CHATEAU PICHON LONGUEVILLE BARON (Pauillac) 3.39 37.50',
 '893 CHATEAU LEOVILLE POYFERRE (St Julien) 3.49 38.50',
 '759 CHATEAU MONTROSE (St Estephe) 3.79 41.75',
 '717 CHATEAU BEYCHEVELLE (St Julien) 3.99 43.95',
 '711 CHATEAU LEOVILLE LASCASES (St Julien) 3.99 43.95',
 '714 CHATEAU PALMER (Margaux) 3.99 43.95',
 '140 CHATEAU LEOVILLE LASCASES (St Julien) 2.99 32.95',
 '107 CHATEAU HAUT BATAILLEY (Pauillac) 2.99 32.95',
 '197 CHATEAU LA POINTE (Pomerol) 2.99 32.95',
 '261 CHATEAU CLOS FOURTET (St Emilion) 2.99 32.95',
 '207 CHATEAU PICHON LONGUEVILLE BARON (Pauillac) 3.49 38.50',
 '115 CHATEAU LAFITE ROTHSCHILD (Pauillac) 7.99 87.50',
 '250 CHATEAU LATOUR POMEROL 1953 3.99 43.95',
